In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/humpback-whale-identification/sample_submission.csv
/kaggle/input/humpback-whale-identification/train.csv
/kaggle/input/humpback-whale-identification/test/fc5634f1c.jpg
/kaggle/input/humpback-whale-identification/test/ed344e8e7.jpg
/kaggle/input/humpback-whale-identification/test/8cf0fd984.jpg
/kaggle/input/humpback-whale-identification/test/6513cfc1f.jpg
/kaggle/input/humpback-whale-identification/test/fd940ccca.jpg
/kaggle/input/humpback-whale-identification/test/9e602e86e.jpg
/kaggle/input/humpback-whale-identification/test/7fb06a038.jpg
/kaggle/input/humpback-whale-identification/test/10bf25ffb.jpg
/kaggle/input/humpback-whale-identification/test/6c6c5aec8.jpg
/kaggle/input/humpback-whale-identification/test/f4bb70495.jpg
/kaggle/input/humpback-whale-identification/test/a7c8fbf67.jpg
/kaggle/input/humpback-whale-identification/test/fbe59ed5d.jpg
/kaggle/input/humpback-whale-identification/test/8ceb7e5fc.jpg
/kaggle/input/humpback-whale-identification/test/ba7167678.jp

In [56]:
os.listdir("../input/")

['humpback-whale-identification']

In [57]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Flatten, ZeroPadding2D, Rescaling
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

In [58]:
class Config:
    image_path = '../input/humpback-whale-identification/train/'
    batch_size = 128
    img_height = 224
    img_width = 224
    val_size = int(len(pd.read_csv('../input/humpback-whale-identification/train.csv')) * 0.1)
    epoch = 10
    ft_epoch = 1
    base_learning_rate = 0.0001

In [59]:
train_df = pd.read_csv("/kaggle/input/humpback-whale-identification/train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [60]:
train_df.describe()

,Image,Id
count,25361,25361
unique,25361,5005
top,0000e88ab.jpg,new_whale
freq,1,9664


In [61]:
train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df))
train_ds = train_ds.shuffle(buffer_size=10)
train_ds = train_ds.skip(Config.val_size*6)
val_ds = train_ds.take(Config.val_size)

In [62]:
for batch in train_ds:
    print(batch)
    break

{'Image': <tf.Tensor: shape=(), dtype=string, numpy=b'990e1946f.jpg'>, 'Id': <tf.Tensor: shape=(), dtype=string, numpy=b'w_d3b6c6b'>}


In [63]:
def get_data(item):
    img = tf.io.read_file(Config.image_path+'/'+item['Image'])
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (Config.img_height, Config.img_width))
    
    return img, (item['Id'])

# def get_data_test(item):
#     img = tf.io.read_file(Config.image_path+'/'+item['img_path'])
#     img = tf.io.decode_jpeg(img, channels=3)
#     img = tf.image.resize(img, (Config.img_height, Config.img_width))
#     return img, item['img_path']

In [64]:
train_ds = train_ds.map(get_data)
val_ds = val_ds.map(get_data)

In [65]:
train_ds = train_ds.cache()
val_ds = val_ds.cache()

In [66]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.batch(Config.batch_size)
train_ds = train_ds.prefetch(AUTOTUNE)
train_ds = train_ds.repeat(Config.batch_size)

val_ds = val_ds.batch(Config.batch_size)
val_ds = val_ds.prefetch(AUTOTUNE)
val_ds = val_ds.repeat(Config.batch_size)

# test_ds = test_ds.batch(Config.batch_size)
# test_ds = test_ds.prefetch(AUTOTUNE)

In [67]:
num_classes = 5005
model = tf.keras.Sequential([
    Rescaling(1./255),
    ZeroPadding2D(3),
    Conv2D(32, (7,7),1, activation='relu'),
    BatchNormalization(axis=3, momentum = 0.99, epsilon=0.001),
    MaxPool2D(),
    
    Conv2D(64, (7,7),1, activation='relu'),
    BatchNormalization(axis=3, momentum = 0.99, epsilon=0.001),
    MaxPool2D(),
    Flatten(),
    
    Dense(num_classes)
])


In [68]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[tf.keras.metrics.TopKCategoricalAccuracy()])


In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs = 10
)